In [85]:
%pip install beautifulsoup4 dateparser openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 3.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import requests
from bs4 import BeautifulSoup

response = requests.get('https://www.icj-cij.org/fr/decisions?type=1&from=1946&to=2023&sort_bef_combine=order_DESC')

if not response.ok:
    print('Error in request')
    exit()

soup = BeautifulSoup(response.content)
rows = soup.find_all("div", {"class": "views-row"})

In [24]:
len(rows)

807

In [59]:
from dateparser.search import search_dates

row = rows[0]
articles = []

for row in rows:
    title = row.find('div', {'class': 'views-field-field-document-long-title'}).text.strip()
    subtitle = row.find('div', {'class': 'views-field-field-document-long-title-1'}).text.strip()
    content = row.find('div', {'class': 'views-field-field-case-long-title'}).text.strip()
    footer = row.find('div', {'class': 'views-field-field-icj-document-subtitle'}).text.strip()

    date = search_dates(title)[0][1]

    blacklist = ['Recevabilité', 'Prorogation de délai', 'Fixation de délai', "Décision sur l'organisation des audiences publiques"]

    if any([black_token in footer for black_token in blacklist]):
        continue

    exception = 'Arrêt' in title and 'Exception préliminaire' in footer
    competence = 'Arrêt' in title and 'Compétence de la Cour' in footer

    if 'Arrêt' in title:
        type = 'Arrêt'
    elif 'Ordonnance' in title:
        type = 'Ordonnance'
    elif 'Avis consultatif' in title:
        type = 'Avis consultatif'
    else:
        type = 'Autre'

    ask = 'Demande en indication' in footer

    article = [
        title,
        date,
        subtitle,
        content,
        footer,
        exception,
        competence,
        type,
        ask
    ]

    articles.append(article)

In [127]:
import pandas as pd


columns = ['title', 'date', 'subtitle', 'content', 'footer', 'exception', 'competence', 'type', 'ask']
df = pd.DataFrame(articles, columns=columns)

In [128]:
total_unique_arret = df[df['type'] == 'Arrêt'].groupby('content').agg('count').count().title
total_arret = df[df['type'] == 'Arrêt'].count().title

print(f'Il y a {total_arret} Arrêts dont {total_unique_arret} uniques.')

Il y a 150 Arrêts dont 115 uniques.


In [129]:
total_unique_ordonnance = df[(df['type'] == 'Ordonnance') & (df['ask'] == True)].groupby('content').agg('count').count().title
total_ordonnance = df[(df['type'] == 'Ordonnance') & (df['ask'] == True)].count().title

print(f'Il y a {total_ordonnance} ordonnances dont {total_unique_ordonnance} uniques.')

Il y a 53 ordonnances dont 49 uniques.


In [130]:
total_unique_arret = df[df['type'] == 'Avis consultatif'].groupby('content').agg('count').count().title
total_arret = df[df['type'] == 'Avis consultatif'].count().title

print(f'Il y a {total_arret} Avis consultatif dont {total_unique_arret} uniques.')

Il y a 27 Avis consultatif dont 26 uniques.


In [131]:
df[(df['type'] == 'Arrêt') & ((df['competence'] == True) | (df['exception'] == True))].groupby(['content', 'date', 'competence', 'exception']).agg('count')

,,,,title,subtitle,footer,type,ask
content,date,competence,exception,,,,,
Actions armées frontalières et transfrontalières (Nicaragua c. Honduras),1988-12-20,True,False,1,1,1,1,1
Activités armées sur le territoire du Congo (nouvelle requête: 2002) (République démocratique du Congo c. Rwanda),2006-02-03,True,False,1,1,1,1,1
Activités militaires et paramilitaires au Nicaragua et contre celui-ci (Nicaragua c. Etats-Unis d'Amérique),1984-11-26,True,False,1,1,1,1,1
Ambatielos (Grèce c. Royaume-Uni),1952-07-01,False,True,1,1,1,1,1
Anglo-Iranian Oil Co. (Royaume-Uni c. Iran),1952-07-22,False,True,1,1,1,1,1
Compétence en matière de pêcheries (Espagne c. Canada),1998-12-04,True,False,1,1,1,1,1
Compétence en matière de pêcheries (Royaume-Uni c. Islande),1973-02-02,True,False,1,1,1,1,1
Compétence en matière de pêcheries (République fédérale d'Allemagne c. Islande),1973-02-02,True,False,1,1,1,1,1
Détroit de Corfou (Royaume-Uni de Grande-Bretagne et d'Irlande du Nord c. Albanie),1948-03-25,False,True,1,1,1,1,1


In [132]:
df_type = df[(df['type'] != 'Ordonnance') | (df['ask'] == True)].groupby(['type', 'ask']).agg(count=('title', 'count'))
df_type

,,count
type,ask,
Arrêt,False,150
Autre,False,2
Avis consultatif,False,27
Ordonnance,True,53


In [137]:
import matplotlib.pyplot as plt


df['year'] = df['date'].dt.year.astype('int')

df_year = df[(df['type'] != 'Ordonnance') | (df['ask'] == True)]
df_year = df_year.pivot_table(index='year', columns='type', values='title', aggfunc='count', fill_value=0)

In [138]:
with pd.ExcelWriter('/Users/guillaume/Desktop/export.xlsx') as writer:
    df.to_excel(writer, sheet_name='Raw Dataset')
    df_type.to_excel(writer, sheet_name='Count per type')
    df_year.to_excel(writer, sheet_name='Type per year')